In [14]:
from satellite_analysis.graphs import massrelation as massrelation
from satellite_analysis.massrelations import stellarmass_relations
import yt
import glob
import numpy as np
import matplotlib.pyplot as plt

#first find all of the halomass catalogs

#add argparse and make the out_dir if it doesnt exits
output_dir = '/Users/user1/documents/GEN3/stellarmassrelation/rockstarytcomparison/fixed15'
halomass_dirs = glob.glob('/Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/*')

#now, find the earliest and latest snapshots for each catalog
earliest_snap = []
latest_snap = []
for directory in halomass_dirs:
    all_snaps = glob.glob('%s/*.ascii' % directory)
    all_snaps.sort()
    earliest_snap.append(all_snaps[0])
    latest_snap.append(all_snaps[-1])

earliest_snap = earliest_snap[0][-9:-6]
latest_snap = latest_snap[-1][-9:-6]

#now make the list of all possible snapshots from the earlies and latest snap
a_list = [int(earliest_snap) + 10*x for x in range((int(latest_snap) - int(earliest_snap))//10 + 1)]

#now now I set up a nested loop to iterate over each a in my a_list in each of my VELA_dir halomass folder
for a in a_list:
    print('Generating dataset for a: %s' % a)
    
    central_ids, satellite_ids = [], []
    Mvir, Mpeak = [], []
    central_stellarmass010rvir, satellite_stellarmass010rvir, = [], []
    central_darkmattermass010rvir, satellite_darkmattermass010rvir, = [], []
    central_ratio010rvir, satellite_ratio010rvir = [], []
    central_stellarmass015rvir, satellite_stellarmass015rvir, = [], []
    central_darkmattermass015rvir, satellite_darkmattermass015rvir, = [], []
    central_ratio015rvir, satellite_ratio015rvir = [], []
    central_stellarmass020rvir, satellite_stellarmass020rvir, = [], []
    central_darkmattermass020rvir, satellite_darkmattermass020rvir, = [], []
    central_ratio020rvir, satellite_ratio020rvir = [], []
    central_stellarmassrvir, satellite_stellarmassrvir, = [], []
    central_darkmattermassrvir, satellite_darkmattermassrvir, = [], []
    central_ratiorvir, satellite_ratiorvir = [], []
    
    for halomass_dir in halomass_dirs:
        print('Trying file: %s/halomass%s.ascii' % (halomass_dir, a))
        try:
            readfile = open('%s/halomass%s.ascii' % (halomass_dir, a))
            lines = readfile.readlines()
            halomass_lines = []
            for line in lines:
                halomass_lines.append(line.split())
            del halomass_lines[0]
            for data_lists in halomass_lines:
                #filter out the satellites (pid =/= -1) from the central galaxies they orbit (pid = -1)
                if float(data_lists[1]) == -1:
                    central_ids.append(float(data_lists[0]))
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    Mvir.append(float(data_lists[2]))
                    central_stellarmass010rvir.append(float(data_lists[4])*.7)
                    central_darkmattermass010rvir.append(float(data_lists[5]))
                    central_ratio010rvir.append(float(data_lists[4])/float(data_lists[2])*.7)
                    central_stellarmass015rvir.append(float(data_lists[6])*.7)
                    central_darkmattermass015rvir.append(float(data_lists[7]))
                    central_ratio015rvir.append(float(data_lists[6])/float(data_lists[2])*.7)
                    central_stellarmass020rvir.append(float(data_lists[8])*.7)
                    central_darkmattermass020rvir.append(float(data_lists[9])*.7)
                    central_ratio020rvir.append(float(data_lists[8])/float(data_lists[2])*.7)
                    central_stellarmassrvir.append(float(data_lists[10])*.7)
                    central_darkmattermassrvir.append(float(data_lists[11]))
                    central_ratiorvir.append(float(data_lists[10])/float(data_lists[2])*.7)
                else:
                    satellite_ids.append(float(data_lists[0]))
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    Mpeak.append(float(data_lists[3])*.7)
                    satellite_stellarmass010rvir.append(float(data_lists[4])*.7)
                    satellite_darkmattermass010rvir.append(float(data_lists[5])*.7)
                    satellite_ratio010rvir.append(float(data_lists[4])/float(data_lists[3])*.7)
                    satellite_stellarmass015rvir.append(float(data_lists[6])*.7)
                    satellite_darkmattermass015rvir.append(float(data_lists[7])*.7)
                    satellite_ratio015rvir.append(float(data_lists[6])/float(data_lists[3])*.7)
                    satellite_stellarmass020rvir.append(float(data_lists[8])*.7)
                    satellite_darkmattermass020rvir.append(float(data_lists[9])*.7)
                    satellite_ratio020rvir.append(float(data_lists[8])/float(data_lists[3])*.7)
                    satellite_stellarmassrvir.append(float(data_lists[10])*.7)
                    satellite_darkmattermassrvir.append(float(data_lists[11])*.7)
                    satellite_ratiorvir.append(float(data_lists[10])/float(data_lists[3])*.7)
            readfile.close()
        except FileNotFoundError:
            print('No file: %s/halomass%s.ascii' % (halomass_dir, a))
    Mvir = [x/10**12 for x in Mvir]
    central_darkmattermassrvir = [x/10**12 for x in central_darkmattermassrvir]
    difference = []
    for x in range(len(Mvir)):
        difference.append(Mvir[x]-central_darkmattermassrvir[x])
    plt.figure(1, figsize=(20,10))
    plt.subplot(121)
    plt.title('Scale Factor: {}'.format(a))
    plt.xlabel('Rockstar Mvir (Msun/h)')
    plt.ylabel('YT Mvir (Msun/h)')
    #plt.xscale('log')
    #plt.yscale('log')
    largest_mvir_rockstar = np.amax(Mvir)
    largest_mvir_yt = np.amax(central_darkmattermassrvir)
    smallest_mvir_rockstar = np.amin(Mvir)
    smallest_mvir_yt = np.amin(central_darkmattermassrvir)
    plt.xlim(smallest_mvir_rockstar-.01, largest_mvir_rockstar+.01)
    plt.ylim(smallest_mvir_yt-.01, largest_mvir_yt+.01)
    plt.scatter(Mvir, central_darkmattermassrvir, s = 5)
    x = np.linspace(0, largest_mvir_rockstar+100, largest_mvir_rockstar+100)
    plt.plot(x, x, '--m')
    
    plt.subplot(122)
    plt.xlabel('Rockstar Mvir')
    plt.ylabel('Difference')
    plt.scatter(Mvir, difference)
    plt.savefig('{}/{}.png'.format(output_dir, a))
    plt.close()
        

Generating dataset for a: 100
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/VELA15/halomass100.ascii


/Users/user1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


Generating dataset for a: 110
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/VELA15/halomass110.ascii
Generating dataset for a: 120
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/VELA15/halomass120.ascii
Generating dataset for a: 130
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/VELA15/halomass130.ascii
Generating dataset for a: 140
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/VELA15/halomass140.ascii
Generating dataset for a: 150
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/VELA15/halomass150.ascii
Generating dataset for a: 160
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/VELA15/halomass160.ascii
Generating dataset for a: 170
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/VELA15/halomass170.ascii
Generating dataset for a: 180
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/fixedVELA15/V

In [7]:
from satellite_analysis.catalogreaders import consistentcatalogreader as consistent

input_dir = '/Users/user1/documents/GEN3/rockstar001analysis/VELA07/hlists'

consistent.consistent_catalog_reader(input_dir, remove_subhalos='True')

Consistent File Indices: ['100', '110', '120', '130', '140', '150', '160', '170', '180', '190', '210', '220', '230', '240', '250', '260', '270', '280', '290', '300', '310', '320', '330', '340', '350', '360', '370', '380', '390', '400', '410', '420', '430', '440', '450', '460', '470', '480', '490', '500', '510', '520', '530', '540']

Using Indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]

Number of Halos found for snapshot 0 : 2
Number of Halos found for snapshot 1 : 4
Number of Halos found for snapshot 2 : 5
Number of Halos found for snapshot 3 : 85
Number of Halos found for snapshot 4 : 70
Number of Halos found for snapshot 5 : 106
Number of Halos found for snapshot 6 : 269
Number of Halos found for snapshot 7 : 153
Number of Halos found for snapshot 8 : 209
Number of Halos found for snapshot 9 : 204
Number of Halos found for snapshot 10 : 265
Number of Halos 

In [11]:
for index in consistent.snapshot_index:
    print('Scale: {}'.format(consistent.consistent_file_index[index]))
    print('Mvir: {}'.format(float(consistent.halo_data_sorted[index][0][10])/.7))
    print('Rvir: {}'.format(float(consistent.halo_data_sorted[index][0][11])/.7))
    print('')

Scale: 100
Mvir: 2870000000000.0
Rvir: 471.58142857142866

Scale: 110
Mvir: 2708571428571.4287
Rvir: 462.50857142857143

Scale: 120
Mvir: 2828571428571.4287
Rvir: 469.1785714285715

Scale: 130
Mvir: 2801428571428.572
Rvir: 467.53857142857146

Scale: 140
Mvir: 2774285714285.7144
Rvir: 465.88714285714286

Scale: 150
Mvir: 2550000000000.0
Rvir: 452.8914285714286

Scale: 160
Mvir: 2532857142857.143
Rvir: 451.6828571428572

Scale: 170
Mvir: 2515714285714.2856
Rvir: 450.4671428571429

Scale: 180
Mvir: 2342857142857.143
Rvir: 439.7585714285715

Scale: 190
Mvir: 2332857142857.143
Rvir: 438.9157142857143

Scale: 210
Mvir: 2157142857142.8572
Rvir: 427.05857142857144

Scale: 220
Mvir: 2041428571428.5715
Rvir: 419.00857142857143

Scale: 230
Mvir: 865857142857.143
Rvir: 314.63142857142856

Scale: 240
Mvir: 942571428571.4286
Rvir: 323.39714285714285

Scale: 250
Mvir: 1882857142857.143
Rvir: 406.9271428571429

Scale: 260
Mvir: 1858571428571.4287
Rvir: 404.8357142857143

Scale: 270
Mvir: 1830000000000

In [5]:
for index in range(len(consistent.halo_data_sorted[24])):
    print(consistent.halo_data_sorted[24][index])

['0.34034', '1122549', '0.35008', '1141398', '1188', '-1', '-1', '-1', '0', '1.03226e+12', '4.48000e+11', '196.732000', '8.274000', '156.400000', '1', '0.10024', '254.360000', '6.30536', '6.73423', '6.80326', '191.990', '21.300', '-1.090', '-3.315e+10', '-1.803e+10', '-3.910e+10', '0.01192', '33665', '31', '1435475', '13110', '24', '308928', '308927', '55', '0.00000', '-1', '8.50271', '447970000000', '441620000000', '430680000000', '326850000000', '256150000000', '20.0391', '9.59', '0.00755', '0.7835', '0.60495', '-0.84927', '1.30136', '0.47755', '0.72585', '0.67242', '1.11339', '-0.29313', '-0.73445', '0.2497', '579200000000', '190600000000', '51.735', '4.48000e+11', '7.35100e+11', '254.360000', '429.720000', '0.10024', '-3.332e+00', '-3.332e+00', '-3.229e+00', '-2.268e+01', '0.000e+00', '3.745e-11', '8.232e-11', '0.10024', '0.34034', '0.34034', '4.480e+11', '254.360', '429.720', '0.00000', '-0.22774', '-1.00000', '1122549']
['0.34034', '1130796', '0.35008', '1148738', '47', '-1', '-1